# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
# !pip install torchsummaryX wandb --quiet

: 

In [1]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
from sklearn import metrics
import torch.nn.functional as F
import pdb
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

/home/jwa/anaconda3/envs/CMU/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jwa/anaconda3/envs/CMU/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Device:  cuda


In [2]:
def set_random_seed(seed_num=1):
	random.seed(seed_num)
	np.random.seed(seed_num)
	torch.manual_seed(seed_num)
	torch.cuda.manual_seed(seed_num)
	torch.cuda.manual_seed_all(seed_num)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False
set_random_seed(seed_num=1)

In [3]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [5]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"younkyungjwa","key":"7703a5bdf09ec9579445480e8e2b34c9"}') 
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

# !chmod 600 /root/.kaggle/kaggle.json
import os

# 닉네임, 토큰 설정

os.environ["KAGGLE_USERNAME"] = 'younkyungjwa'
os.environ["KAGGLE_KEY"] = "7703a5bdf09ec9579445480e8e2b34c9"

In [12]:
# commands to download data from kaggle

# !kaggle competitions download -c 11-785-f22-hw1p2
# !mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d 'data'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [13]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train-clean-100", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        self.mfcc_dir = self.data_path + partition + '/mfcc/'  # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
        self.transcript_dir = self.data_path + partition + '/transcript/' # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names = sorted(os.listdir(self.transcript_dir)) # TODO: List files in self.transcript_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir+mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir+transcript_names[i])
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
                                          # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, [(self.context, self.context), (0, 0)], mode='constant') # TODO

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        self.phonemes_dic = {k: v for v, k in enumerate(self.phonemes)}
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        self.transcripts = [self.phonemes_dic[i] for i in self.transcripts] # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.transcripts)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind+2*self.context+1] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])       

        return frames, phoneme

In [14]:
class AudioTestDataset(torch.utils.data.Dataset):
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, data_path, context, offset=0, partition= "test-clean", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        self.mfcc_dir = self.data_path + partition + '/mfcc/'  # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        self.mfccs= []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir+mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, [(self.context, self.context), (0, 0)], mode='constant') # TODO

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs) - 2*self.context

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind+2*self.context+1] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors

        return frames


# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [15]:
config = {
    'epochs': 15,
    'batch_size' : 2048,
    'context' : 50,
    'learning_rate' : 0.001,
    'architecture' : 'very-low-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [16]:
train_data = AudioDataset('data/',config['context'],offset=0,partition= "train-clean-100") #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset('data/',config['context'],offset=0,partition= "dev-clean") # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset('data/',config['context'],offset=0,partition= "test-clean") # TODO: Create a dataset object using the AudioTestDataset class for the test data 


In [17]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  2048
Context:  50
Input size:  1515
Output symbols:  42
Train dataset samples = 36191134, batches = 17672
Validation dataset samples = 1937496, batches = 947
Test dataset samples = 1943253, batches = 949


In [18]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1515]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [19]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case?
        output_size = 40 #Why?

        layer1_size = 2048  
        layer2_size = 2048
        layer3_size = 2048
        layer4_size = 2048
        layer5_size = 512

        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, layer1_size)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(layer1_size, layer2_size)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(layer2_size, layer3_size)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(layer3_size, layer4_size)
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.Linear(layer4_size, layer5_size)
        )        
        self.layer6 = torch.nn.Sequential(
            torch.nn.Linear(layer5_size, output_size)
        )
        self.bn1 = torch.nn.BatchNorm1d(layer1_size)  
        self.bn2 = torch.nn.BatchNorm1d(layer2_size)  
        self.bn3 = torch.nn.BatchNorm1d(layer3_size)  
        self.bn4 = torch.nn.BatchNorm1d(layer4_size)  
        self.bn5 = torch.nn.BatchNorm1d(layer5_size)  

        self.dropout1 = torch.nn.Dropout(0.2)
        self.dropout2 = torch.nn.Dropout(0.2)
        

    def forward(self, x):
        #pdb.set_trace()
        x = F.relu(self.bn1(self.layer1(x)))
        x = F.relu(self.bn2(self.layer2(x)))
        x = F.relu(self.bn3(self.layer3(x)))
        x = F.relu(self.bn4(self.layer4(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn5(self.layer5(x)))
        x = self.dropout2(x)
        x = self.layer6(x)

        return x

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [20]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                    Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                               
0_layer1.Linear_0   [1515, 2048]  [2048, 2048]  3.104768M   3.10272M
1_bn1                     [2048]  [2048, 2048]     4.096k     2.048k
2_layer2.Linear_0   [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
3_bn2                     [2048]  [2048, 2048]     4.096k     2.048k
4_layer3.Linear_0   [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
5_bn3                     [2048]  [2048, 2048]     4.096k     2.048k
6_layer4.Linear_0   [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
7_bn4                     [2048]  [2048, 2048]     4.096k     2.048k
8_dropout1                     -  [2048, 2048]          -          -
9_layer5.Linear_0    [2048, 512]   [2048, 512]  1.049088M  1.048576M
10_bn5                     [512]   [2048, 512]     1.024k      512.0
11_dropout2                    -   [2048, 512]          -          -
12_layer6.Linear_0     [512, 40]  

/home/jwa/anaconda3/envs/CMU/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_layer1.Linear_0,"[1515, 2048]","[2048, 2048]",3104768.0,3102720.0
1_bn1,[2048],"[2048, 2048]",4096.0,2048.0
2_layer2.Linear_0,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
3_bn2,[2048],"[2048, 2048]",4096.0,2048.0
4_layer3.Linear_0,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
5_bn3,[2048],"[2048, 2048]",4096.0,2048.0
6_layer4.Linear_0,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
7_bn4,[2048],"[2048, 2048]",4096.0,2048.0
8_dropout1,-,"[2048, 2048]",NaN,NaN


In [21]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
#optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.99)
# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [22]:
torch.cuda.empty_cache()
gc.collect()

246

In [23]:
def train(model, optimizer, criterion, dataloader):
    #pdb.set_trace()

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in enumerate(dataloader):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
    
    train_loss /= len(dataloader)
    #pdb.set_trace()
    return train_loss

In [24]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [25]:
wandb.login(key="0699a3c4c17f76e3d85a803c4d7039edb8c3a3d9") #API Key is in your wandb account, under settings (wandb.ai/settings)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jwa/.netrc


True

In [26]:
# Create your wandb run
run = wandb.init(
    name = "third-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

wandb: Currently logged in as: younkyuj. Use `wandb login --relogin` to force relogin


In [27]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/home/jwa/Code/CMU_IDL/HW1P2/wandb/run-20221001_015055-3r53nwe6/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [30]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))
    
    train_loss = train(model, optimizer, criterion, train_loader)
    accuracy = eval(model, val_loader)
    
    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})
    scheduler.step()

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

        ### Save checkpoint with information you want
        '''
        torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
                'acc': accuracy}, 
          './model_checkpoint.pth')
        '''
        torch.save(model.state_dict(), 'weights/model.pt')
      
        best_acc = accuracy
    else:
        pass#scheduler.step()
    
    ### Save checkpoint in wandb
    #wandb.save('checkpoint.pth')

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/15
	Train Loss: 0.4118
	Validation Accuracy: 81.71%

Epoch 2/15
	Train Loss: 0.3357
	Validation Accuracy: 82.02%

Epoch 3/15
	Train Loss: 0.2863
	Validation Accuracy: 81.89%

Epoch 4/15
	Train Loss: 0.2505
	Validation Accuracy: 81.77%

Epoch 5/15
	Train Loss: 0.2245
	Validation Accuracy: 81.74%

Epoch 6/15
	Train Loss: 0.2042
	Validation Accuracy: 81.69%

Epoch 7/15
	Train Loss: 0.1880
	Validation Accuracy: 81.56%

Epoch 8/15
	Train Loss: 0.1739
	Validation Accuracy: 81.41%

Epoch 9/15
	Train Loss: 0.1624
	Validation Accuracy: 81.33%

Epoch 10/15
	Train Loss: 0.1528
	Validation Accuracy: 81.32%

Epoch 11/15
	Train Loss: 0.1444
	Validation Accuracy: 81.26%

Epoch 12/15
	Train Loss: 0.1364
	Validation Accuracy: 81.22%

Epoch 13/15
	Train Loss: 0.1299
	Validation Accuracy: 81.17%

Epoch 14/15
	Train Loss: 0.1240
	Validation Accuracy: 81.19%

Epoch 15/15
	Train Loss: 0.1187
	Validation Accuracy: 81.14%


train loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁
validation accuracy,▁▇█▇▇▇▇▆▆▅▅▅▅▅▅▄
train loss,0.11871
validation accuracy,81.13643


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad():

        for i, frames in enumerate(tqdm(test_loader)):

            frames = frames.float().to(device) 
            output = model(frames)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(output, dim=1).cpu().tolist()
            test_predictions.extend(predicted_phonemes)
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 

            del frames, output
            torch.cuda.empty_cache()

    return test_predictions

In [ ]:
model.load_state_dict(torch.load('weights/model.pt',  map_location=device))
predictions = test(model, test_loader)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"